In [1]:
import numpy as np
import cv2
import glob

In [2]:
dataset_dir = "/home/petrus/Downloads/SpillData/Stagnant water and Wet surface Dataset"
images = glob.glob(dataset_dir+'/*.jpeg')

In [16]:
for ix,img_path in enumerate(images):
    img = cv2.imread(img_path)
    img_h,img_w,_ = img.shape
    with open(img_path[:-5]+'.txt','r') as fp:
        annots = fp.read().splitlines()
    
    puddles = []
    wet_surface = []
    for annot in annots:
        cat,xs,ys,ws,hs = annot.split(' ')
        x = int(float(xs)*img_w)
        y = int(float(ys)*img_h)
        w = int(float(ws)*img_w)
        h = int(float(hs)*img_w)
        #if cat == '0':
        #    cv2.rectangle(img, (x-w//2, y-h//2), (x+w//2, y+h//2), (0,255,0), 2)
        #elif cat == '1':
        #    cv2.rectangle(img, (x-w//2, y-h//2), (x+w//2, y+h//2), (255,0,0), 2)
        
        if cat == '0':
            puddles.append(np.array([x-w//2, y-h//2, x+w//2, y+h//2]))
        
        wet_surface.append(np.array([x-w//2, y-h//2, x+w//2, y+h//2]))
    
    if len(wet_surface) == 0:
        continue
        
    wet_surface = np.stack(wet_surface)
    p_mins = wet_surface.min(axis=0)
    p_maxs = wet_surface.max(axis=0)
    if p_mins[0] > 70:
        neg_crop = img[:,:p_mins[0]-20]
        cv2.imwrite("/home/petrus/Downloads/SpillData/puddle_crops/Not_Puddle/{}xl.jpeg".format(ix),neg_crop)
    if p_mins[1] > 70:
        neg_crop = img[:p_mins[1]-20,:]
        cv2.imwrite("/home/petrus/Downloads/SpillData/puddle_crops/Not_Puddle/{}yl.jpeg".format(ix),neg_crop)
    if img_w-p_maxs[2] > 70:
        neg_crop = img[:,p_maxs[2]+20:]
        cv2.imwrite("/home/petrus/Downloads/SpillData/puddle_crops/Not_Puddle/{}xg.jpeg".format(ix),neg_crop)
    if img_h-p_maxs[3] > 70:
        neg_crop = img[p_maxs[3]+20:,:]
        cv2.imwrite("/home/petrus/Downloads/SpillData/puddle_crops/Not_Puddle/{}yg.jpeg".format(ix),neg_crop)
    
    if len(puddles) == 0:
        continue
        
    puddles = np.stack(puddles)
    p_mins = puddles.min(axis=0)
    p_maxs = puddles.max(axis=0)
    crop_bbox = [p_mins[0],p_mins[1],p_maxs[2],p_maxs[3]]
    crop_img = img[crop_bbox[1]:crop_bbox[3],crop_bbox[0]:crop_bbox[2]]
    
    cv2.imwrite("/home/petrus/Downloads/SpillData/puddle_crops/Puddle/{}.jpeg".format(ix),crop_img)
    
    #cv2.imshow('a',img)
    #if cv2.waitKey(0) & 0xFF == ord('q'):
    #    break

#cv2.destroyAllWindows()